### **In this notebook an autoencoder with most of encoder layers pretrained on CIFAR dataset will be created.**
### The goal is:
* import MNIST and train autoencoder
* OR download pretrained lightning model from [Amsterdam University](https://uvadlc-notebooks.rtfd.io)
* Make the sceleton of autoencoder, that will distinguish between L-mode, H-mode and ELM using images and .csv from COMPASS
* **profit**

In [1]:

import os
import urllib.request
from urllib.error import HTTPError
import numpy as np
from PIL import Image
import pytorch_lightning as pl
import matplotlib
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
from pytorch_lightning.callbacks import Callback, LearningRateMonitor, ModelCheckpoint
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torchvision.datasets import CIFAR10
from tqdm.notebook import tqdm

%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg", "pdf")  # For export
matplotlib.rcParams["lines.linewidth"] = 2.0
sns.reset_orig()
sns.set()

# Tensorboard extension (for visualization purposes later)
%load_ext tensorboard

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = os.environ.get("PATH_DATASETS", "data")
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = os.environ.get("PATH_CHECKPOINT", "saved_models/tutorial9")

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Seed set to 42


Device: cpu


### Import CIFAR dataset

Here I'll try to use pretrained in UVA model

In [2]:
latent_dim = 128 
path = f"./saved_models/tutorial9/cifar10_{latent_dim}.ckpt"

In [3]:
# Github URL where saved models are stored for this tutorial
base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial9/"
# Files to download
pretrained_files = ["cifar10_64.ckpt", "cifar10_128.ckpt", "cifar10_256.ckpt", "cifar10_384.ckpt"]
# Create checkpoint path if it doesn't exist yet
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

# For each file, check whether it already exists. If not, try downloading it.
for file_name in pretrained_files:
    file_path = os.path.join(CHECKPOINT_PATH, file_name)
    if not os.path.isfile(file_path):
        file_url = base_url + file_name
        print("Downloading %s..." % file_url)
        try:
            urllib.request.urlretrieve(file_url, file_path)
        except HTTPError as e:
            print(
                "Something went wrong. Please try to download the files manually,"
                " or contact the author with the full output including the following error:\n",
                e,
            )

### Next 3 cells contain encoder, decoder and autoencoder composed by previous two

In [5]:
class Encoder(nn.Module):
    def __init__(self, num_input_channels: int, base_channel_size: int, latent_dim: int, act_fn: object = nn.GELU):
        """
        Args:
           num_input_channels : Number of input channels of the image. For CIFAR, this parameter is 3
           base_channel_size : Number of channels we use in the first convolutional layers. Deeper layers might use a duplicate of it.
           latent_dim : Dimensionality of latent representation z
           act_fn : Activation function used throughout the encoder network
        """
        super().__init__()
        c_hid = base_channel_size
        self.net = nn.Sequential(
            nn.Conv2d(num_input_channels, c_hid, kernel_size=3, padding=1, stride=2),  # 32x32 => 16x16
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(c_hid, 2 * c_hid, kernel_size=3, padding=1, stride=2),  # 16x16 => 8x8
            act_fn(),
            nn.Conv2d(2 * c_hid, 2 * c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(2 * c_hid, 2 * c_hid, kernel_size=3, padding=1, stride=2),  # 8x8 => 4x4
            act_fn(),
            nn.Flatten(),  # Image grid to single feature vector
            nn.Linear(2 * 16 * c_hid, latent_dim),
        )

    def forward(self, x):
        return self.net(x)

In [6]:
class Decoder(nn.Module):
    def __init__(self, num_input_channels: int, base_channel_size: int, latent_dim: int, act_fn: object = nn.GELU):
        """
        Args:
           num_input_channels : Number of channels of the image to reconstruct. For CIFAR, this parameter is 3
           base_channel_size : Number of channels we use in the last convolutional layers. Early layers might use a duplicate of it.
           latent_dim : Dimensionality of latent representation z
           act_fn : Activation function used throughout the decoder network
        """
        super().__init__()
        c_hid = base_channel_size
        self.linear = nn.Sequential(nn.Linear(latent_dim, 2 * 16 * c_hid), act_fn())
        self.net = nn.Sequential(
            nn.ConvTranspose2d(
                2 * c_hid, 2 * c_hid, kernel_size=3, output_padding=1, padding=1, stride=2
            ),  # 4x4 => 8x8
            act_fn(),
            nn.Conv2d(2 * c_hid, 2 * c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.ConvTranspose2d(2 * c_hid, c_hid, kernel_size=3, output_padding=1, padding=1, stride=2),  # 8x8 => 16x16
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.ConvTranspose2d(
                c_hid, num_input_channels, kernel_size=3, output_padding=1, padding=1, stride=2
            ),  # 16x16 => 32x32
            nn.Tanh(),  # The input images is scaled between -1 and 1, hence the output has to be bounded as well
        )

    def forward(self, x):
        x = self.linear(x)
        x = x.reshape(x.shape[0], -1, 4, 4)
        x = self.net(x)
        return x

In [7]:
class Autoencoder(pl.LightningModule):
    def __init__(
        self,
        base_channel_size: int,
        latent_dim: int,
        encoder_class: object = Encoder,
        decoder_class: object = Decoder,
        num_input_channels: int = 3,
        width: int = 32,
        height: int = 32,
    ):
        super().__init__()
        # Saving hyperparameters of autoencoder
        self.save_hyperparameters()
        # Creating encoder and decoder
        self.encoder = encoder_class(num_input_channels, base_channel_size, latent_dim)
        self.decoder = decoder_class(num_input_channels, base_channel_size, latent_dim)
        # Example input array needed for visualizing the graph of the network
        self.example_input_array = torch.zeros(2, num_input_channels, width, height)

    def forward(self, x):
        """The forward function takes in an image and returns the reconstructed image."""
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat

    def _get_reconstruction_loss(self, batch):
        """Given a batch of images, this function returns the reconstruction loss (MSE in our case)"""
        x, _ = batch  # We do not need the labels
        x_hat = self.forward(x)
        loss = F.mse_loss(x, x_hat, reduction="none")
        loss = loss.sum(dim=[1, 2, 3]).mean(dim=[0])
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        # Using a scheduler is optional but can be helpful.
        # The scheduler reduces the LR if the validation performance hasn't improved for the last N epochs
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.2, patience=20, min_lr=5e-5)
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}

    def training_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch)
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch)
        self.log("test_loss", loss)

In [8]:
def compare_imgs(img1, img2, title_prefix=""):
    # Calculate MSE loss between both images
    loss = F.mse_loss(img1, img2, reduction="sum")
    # Plot images for visual comparison
    grid = torchvision.utils.make_grid(torch.stack([img1, img2], dim=0), nrow=2, normalize=True, range=(-1, 1))
    grid = grid.permute(1, 2, 0)
    plt.figure(figsize=(4, 2))
    plt.title(f"{title_prefix} Loss: {loss.item():4.2f}")
    plt.imshow(grid)
    plt.axis("off")
    plt.show()

    
def two_imgs(img1, img2):
    # Plot images for visual comparison
    grid1 = torchvision.utils.make_grid(img1, nrow=1, normalize=True)
    grid1 = grid1.permute(1, 2, 0)
    grid2 = torchvision.utils.make_grid(img2, nrow=1, normalize=True)
    grid2 = grid2.permute(1, 2, 0)
    fig, axs = plt.subplots(2,1,figsize=(10, 5))
    axs[0].imshow(grid1)
    axs[1].imshow(grid2)
    axs[0].axis("off")
    axs[1].axis("off")

### Encoder and decoder 

### Import pretrained model

In [9]:
premodel = Autoencoder.load_from_checkpoint(path) #pretrained model
model = Autoencoder(base_channel_size=32, latent_dim=latent_dim, encoder_class=Encoder, decoder_class=Decoder)


#LH_model = 

Lightning automatically upgraded your loaded checkpoint from v0.9.0 to v2.1.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint saved_models/tutorial9/cifar10_128.ckpt`


In [10]:
dict = model.state_dict()
premodel_dict = premodel.state_dict()

#select 1-th, 2-th, 3-th and 4-th layers of the encoder
new_dict = {key: values for key, values in premodel_dict.items() if \
            np.logical_and('encoder' in key, np.array(['0' in key, '2' in key, '4' in key, '6' in key]).any())}

for name, param in dict.items():
    print(name)
dict.update(new_dict)

model.load_state_dict(dict)

encoder.net.0.weight
encoder.net.0.bias
encoder.net.2.weight
encoder.net.2.bias
encoder.net.4.weight
encoder.net.4.bias
encoder.net.6.weight
encoder.net.6.bias
encoder.net.8.weight
encoder.net.8.bias
encoder.net.11.weight
encoder.net.11.bias
decoder.linear.0.weight
decoder.linear.0.bias
decoder.net.0.weight
decoder.net.0.bias
decoder.net.2.weight
decoder.net.2.bias
decoder.net.4.weight
decoder.net.4.bias
decoder.net.6.weight
decoder.net.6.bias
decoder.net.8.weight
decoder.net.8.bias


<All keys matched successfully>

In [11]:

for preparam, param in zip(premodel.parameters(), model.parameters()):
    if (preparam.to(device)==param.to(device)).all():
        param.requires_grad = False
        print('weights or biases are changed')

weights or biases are changed
weights or biases are changed
weights or biases are changed
weights or biases are changed
weights or biases are changed
weights or biases are changed
weights or biases are changed
weights or biases are changed


### Load images as torch tensors

In [13]:
import pandas as pd
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
shot = 16534
LorH = pd.read_csv(f'./LHmode-detection-shot{shot}.csv', index_col=0)

In [14]:
image = Image.open('/compass/Shared/Users/bogdanov/vyzkumny_ukol/imgs/RIS1_18057_t= 972.000.png') 
# Define a transform to convert PIL  
# image to a Torch tensor 
img_trans = transforms.ToTensor()
img_tensor = img_trans(image) 

features = LorH['mode']


In [15]:
img_tensor.shape

torch.Size([3, 500, 640])

I'll make a simple model where, the number of channels remains constant as data go through the net (num_channels = 3)

In [16]:
class LH_Encoder(nn.Module):
    def __init__(self, num_input_channels: int, base_channel_size: int, latent_dim: int, act_fn: object = nn.GELU):
        """
        Args:
        
           height : input image height in pixels
           width : input image width in pixels
           num_input_channels : Number of input channels of the image. For CIFAR, this parameter is 3
           base_channel_size : Number of channels we use in the first convolutional layers. Deeper layers might use a duplicate of it.
           latent_dim : Dimensionality of latent representation z
           act_fn : Activation function used throughout the encoder network
        """
        super().__init__()
        c_hid = base_channel_size
        self.net = nn.Sequential(
            nn.Conv2d(num_input_channels, c_hid, kernel_size=3, padding=1, stride=2),  # 500x640 => 250x320
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(c_hid, 2 * c_hid, kernel_size=3, padding=1, stride=2),  # 250x320 => 125x160
            act_fn(),
            nn.Conv2d(2 * c_hid, 2 * c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(2 * c_hid, 2 * c_hid, kernel_size=3, padding=1, stride=5),  # 125x160 => 25*32
            act_fn(),
            nn.Flatten(),  # Image grid to single feature vector
            nn.Linear(25*32, latent_dim), #TODO 63*80? clarify the shape of layers (There must be some arithmetic rules)
        )

    def forward(self, x):
        return self.net(x)

dummy_enc = LH_Encoder(num_input_channels=3, base_channel_size=3, latent_dim=1)


In [17]:
class LH_Decoder(nn.Module):
    def __init__(self, num_input_channels: int, base_channel_size: int, latent_dim: int, act_fn: object = nn.GELU):
        """
        Args:
           num_input_channels : Number of channels of the image to reconstruct. For CIFAR, this parameter is 3
           base_channel_size : Number of channels we use in the last convolutional layers. Early layers might use a duplicate of it.
           latent_dim : Dimensionality of latent representation z
           act_fn : Activation function used throughout the decoder network
        """
        super().__init__()
        c_hid = base_channel_size
        self.linear = nn.Sequential(nn.Linear(latent_dim, 25*32), act_fn()) #TODO 25*32? clarify the shape of layers (There must be some arithmetic rules)
        self.net = nn.Sequential(
            nn.ConvTranspose2d(
                2 * c_hid, 2 * c_hid, kernel_size=3, output_padding=1, padding=1, stride=2
            ),  # 4x4 => 8x8
            act_fn(),
            nn.Conv2d(2 * c_hid, 2 * c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.ConvTranspose2d(2 * c_hid, c_hid, kernel_size=3, output_padding=1, padding=1, stride=2),  # 8x8 => 16x16
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.ConvTranspose2d(
                c_hid, num_input_channels, kernel_size=3, output_padding=4, padding=1, stride=5
            ),  # 16x16 => 32x32
            nn.Tanh(),  # The input images is scaled between -1 and 1, hence the output has to be bounded as well
        )

    def forward(self, x):
        x = self.linear(x)
        x = x.reshape(x.shape[0], 25, 32)
        x = self.net(x)
        return x

dummy_dec = LH_Decoder(num_input_channels=3, base_channel_size=3, latent_dim=128)

In [18]:
encoded = dummy_enc(img_tensor)
dummy_enc(img_tensor).shape
dummy_dec(encoded).shape


RuntimeError: mat1 and mat2 shapes cannot be multiplied (6x1 and 128x800)

In [19]:
dummy_AE = Autoencoder(base_channel_size=3,latent_dim=128,encoder_class=LH_Encoder, decoder_class=LH_Decoder)

In [35]:
dummy_AE(img_tensor).shape

torch.Size([3, 500, 640])

### create encoder

In [4]:
class LH_encoder(pl.LightningModule):
    def __init__(self, num_input_channels: int, base_channel_size: int, latent_dim: int, act_fn: object = nn.GELU):
        """
        Args:
        
           height : input image height in pixels
           width : input image width in pixels
           num_input_channels : Number of input channels of the image. For CIFAR, this parameter is 3
           base_channel_size : Number of channels we use in the first convolutional layers. Deeper layers might use a duplicate of it.
           latent_dim : Dimensionality of latent representation z
           act_fn : Activation function used throughout the encoder network
        """
        super().__init__()
        c_hid = base_channel_size
        self.net = nn.Sequential(
            nn.Conv2d(num_input_channels, c_hid, kernel_size=3, padding=1, stride=2),  # 500x640 => 250x320
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(c_hid, 2 * c_hid, kernel_size=3, padding=1, stride=2),  # 250x320 => 125x160
            act_fn(),
            nn.Conv2d(2 * c_hid, 2 * c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(2 * c_hid, 2 * c_hid, kernel_size=3, padding=1, stride=5),  # 125x160 => 25*32
            act_fn(),
            nn.Flatten(),  # Image grid to single feature vector
            nn.Linear(25*32, latent_dim), #TODO 63*80? clarify the shape of layers (There must be some arithmetic rules)
        )

    def forward(self, x):
        return self.net(x)

    def _get_reconstruction_loss(self, batch):
        """Given a batch of images, this function returns the reconstruction loss (MSE in our case)"""
        x, _ = batch  # We do not need the labels
        x_hat = self.forward(x)
        loss = F.mse_loss(x, x_hat, reduction="none")
        loss = loss.sum(dim=[1, 2, 3]).mean(dim=[0])
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        # Using a scheduler is optional but can be helpful.
        # The scheduler reduces the LR if the validation performance hasn't improved for the last N epochs
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.2, patience=20, min_lr=5e-5)
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}

    def training_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch)
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch)
        self.log("test_loss", loss)

In [5]:
img_tensor.size()

NameError: name 'img_tensor' is not defined

In [6]:
dummy_encoder = LH_encoder(num_input_channels=3, base_channel_size=3, latent_dim=1)
dummy_encoder(img_tensor)

NameError: name 'img_tensor' is not defined

In [7]:
dummy_encoder = LH_encoder(num_input_channels=3, base_channel_size=3, latent_dim=1)
dummy_enc_trained, result = train(dummy_encoder, train_dataloader, test_dataloader, val_dataloader)

NameError: name 'train' is not defined

### Create a training/testing database

In [9]:
import pandas as pd
shots = [16769, 16773]#[16534, 16769, 16773, 18057]
shot_df = pd.DataFrame([])

In [10]:
for shot in shots:
    df = pd.read_csv(f'/compass/Shared/Users/bogdanov/vyzkumny_ukol/LHmode-detection-shot{shot}.csv')
    shot_df = pd.concat([shot_df, df], axis=0)

In [11]:
df_copy = shot_df['mode'].copy()

In [12]:
df_copy

0       L-mode
1       L-mode
2       L-mode
3       L-mode
4       L-mode
         ...  
2227    L-mode
2228    L-mode
2229    L-mode
2230    L-mode
2231    L-mode
Name: mode, Length: 4464, dtype: object

In [13]:
df_copy[shot_df['mode']=='L-mode']=0
df_copy[shot_df['mode']=='H-mode']=1
df_copy[shot_df['mode']=='ELM']=2

In [14]:
shot_df['mode'] = df_copy

In [15]:
shot_df

,time,mode,filename
0,950.0,0,imgs/RIS1_16769_t=950.0000000000001.png
1,950.2,0,imgs/RIS1_16769_t=950.2000000000002.png
2,950.4,0,imgs/RIS1_16769_t=950.4000000000001.png
3,950.6,0,imgs/RIS1_16769_t=950.6000000000001.png
4,950.8,0,imgs/RIS1_16769_t=950.8000000000001.png
...,...,...,...
2227,1395.4,0,imgs/RIS1_16773_t=1395.400.png
2228,1395.6,0,imgs/RIS1_16773_t=1395.600.png
2229,1395.8,0,imgs/RIS1_16773_t=1395.800.png
2230,1396.0,0,imgs/RIS1_16773_t=1396.000.png


In [16]:
from torchvision.io import read_image
from torch.utils.data import Dataset
class ImageDataset(Dataset):
    def __init__(self, annotations, img_dir, transform=None, target_transform=None):
        self.img_labels = annotations #pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 'filename'])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    
dataset = ImageDataset(annotations=shot_df[['filename', 'mode']], img_dir='/compass/Shared/Users/bogdanov/vyzkumny_ukol')

In [160]:
dataset = ImageDataset(annotations=shot_df[['filename', 'mode']], img_dir='/compass/Shared/Users/bogdanov/vyzkumny_ukol')
dataset.__getitem__(idx=3425)

(tensor([[[1, 2, 0,  ..., 0, 0, 1],
          [0, 1, 1,  ..., 0, 1, 1],
          [0, 0, 0,  ..., 2, 0, 6],
          ...,
          [0, 0, 0,  ..., 4, 1, 1],
          [0, 0, 1,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],
 
         [[0, 0, 0,  ..., 0, 0, 0],
          [1, 0, 0,  ..., 0, 0, 1],
          [0, 0, 0,  ..., 1, 0, 2],
          ...,
          [0, 0, 0,  ..., 0, 2, 1],
          [0, 0, 0,  ..., 0, 0, 0],
          [1, 0, 1,  ..., 0, 0, 1]],
 
         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 1,  ..., 0, 1, 0],
          [0, 0, 0,  ..., 0, 1, 1],
          ...,
          [0, 0, 0,  ..., 2, 1, 0],
          [0, 0, 0,  ..., 1, 0, 1],
          [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8),
 1)

In [161]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

### Training the model

During the training, we want to keep track of the learning progress by seeing reconstructions made by our model.
For this, we implement a callback object in PyTorch Lightning which will add reconstructions every $N$ epochs to our tensorboard:

In [147]:
class GenerateCallback(Callback):
    def __init__(self, input_imgs, every_n_epochs=1):
        super().__init__()
        self.input_imgs = input_imgs  # Images to reconstruct during training
        # Only save those images every N epochs (otherwise tensorboard gets quite large)
        self.every_n_epochs = every_n_epochs

    def on_train_epoch_end(self, trainer, pl_module):
        if trainer.current_epoch % self.every_n_epochs == 0:
            # Reconstruct images
            input_imgs = self.input_imgs.to(pl_module.device)
            with torch.no_grad():
                pl_module.eval()
                reconst_imgs = pl_module(input_imgs)
                pl_module.train()
            # Plot and add to tensorboard
            imgs = torch.stack([input_imgs, reconst_imgs], dim=1).flatten(0, 1)
            grid = torchvision.utils.make_grid(imgs, nrow=2, normalize=True, range=(-1, 1))
            trainer.logger.experiment.add_image("Reconstructions", grid, global_step=trainer.global_step)

In [148]:
def train(model, train_loader, test_loader, val_loader):
    # Create a PyTorch Lightning trainer with the generation callback
    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        max_epochs=500,
        
    )
    trainer.fit(model, train_loader, test_loader)
    trainer.logger._log_graph = True  # If True, we plot the computation graph in tensorboard
    trainer.logger._default_hp_metric = None  # Optional logging argument that we don't need

    # Test best model on validation and test set
    val_result = trainer.test(model, dataloaders=val_loader, verbose=False)
    test_result = trainer.test(model, dataloaders=test_loader, verbose=False)
    result = {"test": test_result, "val": val_result}
    return model, result
def get_train_images(num):
    return torch.stack([train_dataset[i][0] for i in range(num)], dim=0)

In [151]:
dummy_enc_trained, result = train(dummy_encoder, train_dataloader, test_dataloader, val_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x4800 and 800x1)

In [62]:
def models_are_eq(model1, model2):
    for (nameA, paramA), (nameB, paramB) in zip(model.named_parameters(), model2.named_parameters()):
        if (paramA == paramB).all():
            print('{} matches {}'.format(nameA, nameB))
        else:
            print('{} does not match {}'.format(nameA, nameB))
    for p1, p2 in zip(model1.parameters(), model2.parameters()):
        if p1.data.ne(p2.data).sum() > 0:
            return False
    return True

models_are_eq(model.to(device), premodel.to(device))

encoder.net.0.weight matches encoder.net.0.weight
encoder.net.0.bias matches encoder.net.0.bias
encoder.net.2.weight matches encoder.net.2.weight
encoder.net.2.bias matches encoder.net.2.bias
encoder.net.4.weight matches encoder.net.4.weight
encoder.net.4.bias matches encoder.net.4.bias
encoder.net.6.weight matches encoder.net.6.weight
encoder.net.6.bias matches encoder.net.6.bias
encoder.net.8.weight does not match encoder.net.8.weight
encoder.net.8.bias does not match encoder.net.8.bias
encoder.net.11.weight does not match encoder.net.11.weight
encoder.net.11.bias does not match encoder.net.11.bias
decoder.linear.0.weight does not match decoder.linear.0.weight
decoder.linear.0.bias does not match decoder.linear.0.bias
decoder.net.0.weight does not match decoder.net.0.weight
decoder.net.0.bias does not match decoder.net.0.bias
decoder.net.2.weight does not match decoder.net.2.weight
decoder.net.2.bias does not match decoder.net.2.bias
decoder.net.4.weight does not match decoder.net.4.

False